In [1]:
import pandas as pd
import random, time
from nba_api.stats.static import players, teams
from nba_api.stats.endpoints import playercareerstats
from nba_api.stats.endpoints import commonteamroster

In [2]:
# # player1 = input('enter player 1: ')
# player1= "Vince Carter"
# player2 = "Draymond Green"

In [4]:
# webscraping player names with selenium + geckodriver
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.options import Options
from selenium.webdriver import Firefox

url = 'https://dribblegame.com/play'
options = Options()
options.add_argument("--headless")
driver = Firefox(options=options)
driver.get(url)
time.sleep(random.uniform(0,1))
elem = driver.find_element(By.ID, "prompt_player_1")
player1 = elem.text
print(f'text: {elem.text}')
# print(f'tag: {elem.tag_name}')
# print(f'parent: {elem.parent}')
# print(f'location: {elem.location}')
# print(f'size: {elem.size}')

elem2 = driver.find_element(By.ID, "prompt_player_2")
player2 = elem2.text
print(f'text: {elem2.text}')

driver.close()

text: Vince Carter
text: Draymond Green


In [5]:
# TODO figure out a better solution for same name players (the gerald henderson problem)
p1Dict = players.find_players_by_full_name(player1)[-1]
# time.sleep(.600)
p2Dict = players.find_players_by_full_name(player2)[-1]
# time.sleep(.600)

p1Career = playercareerstats.PlayerCareerStats(player_id=p1Dict['id']).get_data_frames()
# time.sleep(.600)
p2Career = playercareerstats.PlayerCareerStats(player_id=p2Dict['id']).get_data_frames()
# time.sleep(.600)

In [6]:
p1_teammates = pd.DataFrame(columns=['PLAYER', 'TeamID'])
for season in range(p1Career[0].shape[0]):
    temp_teammates = commonteamroster.CommonTeamRoster(team_id=p1Career[0]['TEAM_ID'][season],
                                                        season=p1Career[0]['SEASON_ID'][season]).get_data_frames()[0]
    time.sleep(.600)
    p1_teammates = pd.concat([p1_teammates, temp_teammates])
p1_teammates.PLAYER.nunique()

208

In [7]:
p2_teammates = pd.DataFrame(columns=['PLAYER', 'TeamID'])
for season in range(p2Career[0].shape[0]):
    temp_teammates = commonteamroster.CommonTeamRoster(team_id=p2Career[0]['TEAM_ID'][season],
                                                        season=p2Career[0]['SEASON_ID'][season]).get_data_frames()[0]
    time.sleep(.600)
    p2_teammates = pd.concat([p2_teammates, temp_teammates])
p2_teammates.PLAYER.nunique()

79

In [8]:
p1_teammates_u = p1_teammates.groupby(['PLAYER'], as_index=False).first()[['PLAYER','TeamID']]
p2_teammates_u = p2_teammates.groupby(['PLAYER'], as_index=False).first()[['PLAYER','TeamID']]
common_teammates = pd.merge(p1_teammates_u, p2_teammates_u, how='inner', on='PLAYER')

teams_to_query = pd.concat([common_teammates['TeamID_x'], common_teammates['TeamID_y']]).unique()
team_list = pd.DataFrame(columns=['TeamID', 'Team Name'])
for t in teams_to_query:
    temp_team_dict = teams.find_team_name_by_id(t)
    temp_team = pd.DataFrame(data={'TeamID':[int(temp_team_dict['id'])],
                                    'Team Name':[temp_team_dict['abbreviation']]})
    team_list = pd.concat([team_list, temp_team])
    time.sleep(.600)

common_teammates = pd.merge(common_teammates, team_list, how='left', left_on='TeamID_x', right_on='TeamID')
common_teammates = pd.merge(common_teammates, team_list, how='left', left_on='TeamID_y', right_on='TeamID', suffixes=('_P1', '_P2'))

common_teammates = common_teammates.rename(columns={'Team Name_P1': f'{player1}',
                                                    'Team Name_P2': f'{player2}'})[['PLAYER', f'{player1}', f'{player2}']]

In [9]:
# common_teammates = pd.merge(p1_teammates_u, p2_teammates_u, how='inner', on='PLAYER')
print(f"{player1} and {player2} have {common_teammates.shape[0]} teammates in common.")
print(common_teammates.set_index('PLAYER'))

Vince Carter and Draymond Green have 5 teammates in common.
                  Vince Carter Draymond Green
PLAYER                                       
Damian Jones               ATL            GSW
JaMychal Green             MEM            GSW
Kent Bazemore              ATL            GSW
Matt Barnes                ORL            GSW
Richard Jefferson          BKN            GSW
